In [17]:
from elasticsearch import Elasticsearch
from datetime import datetime
from flask import Flask, render_template, request
import csv


In [19]:

es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))
es.info()

C:\Users\vijay\AppData\Local\Temp\ipykernel_36848\2575928368.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))


ObjectApiResponse({'name': 'BOOK-GM09OBS762', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'vJxnvQteTUeURso4lvYqKw', 'version': {'number': '8.10.4', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': 'b4a62ac808e886ff032700c391f45f1408b2538c', 'build_date': '2023-10-11T22:04:35.506990650Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
cnbc_documents = list()
guardian_documents = list()
reuters_documents = list()

with open("cnbc_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        cnbc_documents.append(data)


with open("guardian_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        guardian_documents.append(data)


with open("reuters_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        reuters_documents.append(data)


print(cnbc_documents[:10])
print()

print(guardian_documents[:10])
print()

print(reuters_documents[:10])
print()

[{'Headlines': 'Jim Cramer: A better way to invest in the Covid-19 vaccine gold rush', 'Time': ' 7:51  PM ET Fri, 17 July 2020', 'Description': '"Mad Money" host Jim Cramer recommended buying four companies that are supporting vaccine developers.'}, {'Headlines': "Cramer's lightning round: I would own Teradyne", 'Time': ' 7:33  PM ET Fri, 17 July 2020', 'Description': '"Mad Money" host Jim Cramer rings the lightning round bell, which means he\'s giving his answers to callers\' stock questions at rapid speed.'}, {'Headlines': '', 'Time': '', 'Description': ''}, {'Headlines': "Cramer's week ahead: Big week for earnings, even bigger week for vaccines", 'Time': ' 7:25  PM ET Fri, 17 July 2020', 'Description': '"We\'ll pay more for the earnings of the non-Covid companies if The Lancet publishes some good news from AstraZeneca\'s vaccine trial," Cramer said.'}, {'Headlines': 'IQ Capital CEO Keith Bliss says tech and healthcare will rally', 'Time': ' 4:24  PM ET Fri, 17 July 2020', 'Descripti

In [21]:
def index_documents(index, id, document):
    
    resp = es.index(index=index, id=id, document=document)


In [22]:
for i in range(len(cnbc_documents)):
    
    es.index(index = "cnbc_index", id = i + 1, document = cnbc_documents[i])

for i in range(len(guardian_documents)):
    
    es.index(index = "guardian_index", id = i + 1, document = guardian_documents[i])

for i in range(len(reuters_documents)):
    
    es.index(index = "reuters_index", id = i + 1, document = reuters_documents[i])

In [24]:
resp = es.get(index="cnbc_index", id=1)
print(resp['_source'])
print()

resp = es.get(index="guardian_index", id=1)
print(resp['_source'])
print()

resp = es.get(index="reuters_index", id=1)
print(resp['_source'])
print()

{'Headlines': 'Jim Cramer: A better way to invest in the Covid-19 vaccine gold rush', 'Time': ' 7:51  PM ET Fri, 17 July 2020', 'Description': '"Mad Money" host Jim Cramer recommended buying four companies that are supporting vaccine developers.'}

{'Time': '18-Jul-20', 'Headlines': 'Johnson is asking Santa for a Christmas recovery'}

{'Headlines': 'TikTok considers London and other locations for headquarters', 'Time': 'Jul 18 2020', 'Description': 'TikTok has been in discussions with the UK government over the past few months to locate its headquarters in London, a source familiar with the matter said, as part of a strategy to distance itself from its Chinese ownership.'}



In [28]:
def search_query(q):
    
    body = {
        "size": 20,
        "query": {
            "dis_max": {
                "queries": [
                    {"match": {"Headlines": q}},
                    {"match": {"Description": q}}
                ],
                "tie_breaker": 0.3
            }
        }
    }

    resp = es.search(index="cnbc_index", body=body)

    results = []

    for hit in resp['hits']['hits']:
        
        headline = hit["_source"].get('Headlines')
        time = hit["_source"].get('Time')
        description = hit["_source"].get('Description')

        formatted_result = f"Headline: {headline}\nTime: {time}\nDescription: {description}\n\n"

        print(formatted_result)
        
        formatted_result = formatted_result.replace('\n', '<br>')

        results.append(formatted_result)

    return results


In [29]:
search_query("Jim")

Headline: Jim Cramer reveals his top social media stocks
Time:  6:58  PM ET Mon, 20 May 2019
Description: Jim Cramer ranks the stocks of Twitter, Snap, Pinterest and Facebook.


Headline: Earnings season has reached 'maximum disorientation,' Jim Cramer says
Time:  6:34  PM ET Wed, 23 Oct 2019
Description: Jim Cramer breaks down the counterintuitive trading in Chipotle and Caterpillar and instructs investors to buy the dip.


Headline: Jim Cramer: Macy's, Kohl's 'have never felt more irrelevant'
Time:  7:11  PM ET Mon,  9 Dec 2019
Description: "The problem for Macy's and Kohl's and Nordstrom is that ... their position gets worse and worse with every passing year," Jim Cramer says.


Headline: Jim Cramer: Chipotle is a buy at this price
Time:  7:45  PM ET Mon, 20 April 2020
Description: "Mad Money" host Jim Cramer rings the lightning round bell, which means he's giving his answers to callers' stock questions at rapid speed.


Headline: Jim Cramer deciphers the Wall Street lingo behind st

['Headline: Jim Cramer reveals his top social media stocks<br>Time:  6:58  PM ET Mon, 20 May 2019<br>Description: Jim Cramer ranks the stocks of Twitter, Snap, Pinterest and Facebook.<br><br>',
 "Headline: Earnings season has reached 'maximum disorientation,' Jim Cramer says<br>Time:  6:34  PM ET Wed, 23 Oct 2019<br>Description: Jim Cramer breaks down the counterintuitive trading in Chipotle and Caterpillar and instructs investors to buy the dip.<br><br>",
 'Headline: Jim Cramer: Macy\'s, Kohl\'s \'have never felt more irrelevant\'<br>Time:  7:11  PM ET Mon,  9 Dec 2019<br>Description: "The problem for Macy\'s and Kohl\'s and Nordstrom is that ... their position gets worse and worse with every passing year," Jim Cramer says.<br><br>',
 'Headline: Jim Cramer: Chipotle is a buy at this price<br>Time:  7:45  PM ET Mon, 20 April 2020<br>Description: "Mad Money" host Jim Cramer rings the lightning round bell, which means he\'s giving his answers to callers\' stock questions at rapid speed.<

In [30]:
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def homepage():

    results = []

    if request.method == 'POST':

        query = request.form.get('search_query')
        
        results = search(query)
    
    return render_template('test.html', results=results)


def search(query):

    return search_query(query)


app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Nov/2023 12:06:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 12:06:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2023 12:06:13] "POST / HTTP/1.1" 200 -


Headline: Jim Cramer reveals his top social media stocks
Time:  6:58  PM ET Mon, 20 May 2019
Description: Jim Cramer ranks the stocks of Twitter, Snap, Pinterest and Facebook.


Headline: Earnings season has reached 'maximum disorientation,' Jim Cramer says
Time:  6:34  PM ET Wed, 23 Oct 2019
Description: Jim Cramer breaks down the counterintuitive trading in Chipotle and Caterpillar and instructs investors to buy the dip.


Headline: Jim Cramer: Macy's, Kohl's 'have never felt more irrelevant'
Time:  7:11  PM ET Mon,  9 Dec 2019
Description: "The problem for Macy's and Kohl's and Nordstrom is that ... their position gets worse and worse with every passing year," Jim Cramer says.


Headline: Jim Cramer: Chipotle is a buy at this price
Time:  7:45  PM ET Mon, 20 April 2020
Description: "Mad Money" host Jim Cramer rings the lightning round bell, which means he's giving his answers to callers' stock questions at rapid speed.


Headline: Jim Cramer deciphers the Wall Street lingo behind st

127.0.0.1 - - [28/Nov/2023 12:07:55] "POST / HTTP/1.1" 200 -


Headline: Everything Jim Cramer said on 'Mad Money,' including stock valuations, Ollie's Bargain stock
Time:  8:20  PM ET Thu, 13 Feb 2020
Description: CNBC's Jim Cramer breaks down how objective and subjective valuations are influencing the stock market. He issues a buy call on Ollie's Bargain Outlet stock.


Headline: Cramer: 5 reasons the stock market rebounded
Time:  7:03  PM ET Wed, 30 May 2018
Description: The stock market rebound on Wednesday was yet another example of panic being replaced by rationality, says "Mad Money" host Jim Cramer.


Headline: Cramer Remix: Stay away from Monster Beverage's stock
Time:  8:11  PM ET Fri, 29 March 2019
Description: Jim Cramer breaks down why Monster Beverage is a battleground stock that you don't need.


Headline: Cramer's lightning round: AT&T's stock is an opportunity here
Time:  7:07  PM ET Thu,  3 Jan 2019
Description: Jim Cramer flies through his responses to callers' stock questions, including one on a telecommunications stock with a 